In [1]:
# tampering with the kwd file on passaro
# the /experiment folder is on the ssd of passaro (/usr/local/experiment/raw_data)
%matplotlib inline
from __future__ import division
import pdb
import sys
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
import h5py
from scipy.signal import hilbert
import glob
import errno    
import os
import shutil as sh
import yaml
import logging

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

# Check wihic computer to decide where the things are mounted
comp_name=socket.gethostname()
logger.info('Computer: ' + comp_name)
if  comp_name == 'chim':
    #sys.path.append('/Users/zeke/experiment/ephysDataManagement/ephysScripts')
    experiment_folder = os.path.join(os.path.abspath('W:') , "earneodo", "bci_zf")
elif  'lookfar' in comp_name:
    repos_folder = os.path.abspath('/Users/zeke/reposs')
    experiment_folder = os.path.join(os.path.abspath('/Volumes'), 
                                     'gentner', 
                                     'earneodo', 
                                     'bci_zf')

elif 'lintu' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')

elif 'passaro' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    cube_experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')
    store_experiment_folder = os.path.join('/Data/bci_zf/')
    experiment_folder = os.path.join('/usr/local/experiment')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))
sys.path.append(os.path.join(repos_folder, 'swissknife'))

#import soundtools as st
from scipy.io import wavfile
from bci.core import expstruct as et
from bci import supersession as ss
from bci import stimalign as sa

logger.info('All modules loaded')





2017-03-12 12:31:36,258 root         INFO     Computer: passaro
2017-03-12 12:31:36,682 root         INFO     All modules loaded


In [53]:
bird_id = 'b1114'
sess_day = 'anesth_surface_depth_2017-03-10_16-30-53'
depth = '450'
raw_location = 'rw'
# Make the super session and gather the list of sessions
# get the sessions list of files
raw_data_folder = et.file_names(bird_id)['folders'][raw_location]
sessions = glob.glob(os.path.join(raw_data_folder, sess_day + '*' + str(depth)))
day_par_file_path = os.path.join(sessions[0], et.file_names(bird_id)['structure']['par'])
sess_par = et.get_parameters(bird_id, os.path.split(sessions[0])[-1], location=raw_location)
super_sess_par = sess_par.copy()
data_processor = sess_par['rec_config']['processors']['data']
experiments = ss.list_flatten([glob.glob(os.path.join(s, '*_{}.raw.kwd'.format(data_processor)))[:] for s in sessions ])
experiments.sort()

In [45]:
reload(ss)
ss.process_awake_recording(bird_id, sess_day, depth, raw_location=raw_location)

IOError: [Errno 2] No such file or directory: '/mnt/cube/earneodo/bci_zf/raw_data/b1114/anesth_surface_2017-03-10_15-02-38_1/experiment.par.yml'

In [92]:
raw_location == 'raw'

True

In [73]:
from bci import stimalign as sa

In [74]:
from bci.core.file import h5_functions as h5f

In [54]:
# make the supersession file
super_sess_name = 'day-' + sess_day + '_' + depth
fn = et.file_names(bird_id, super_sess_name, 0)
super_sess_path = fn['folders']['ss']
super_file_path = os.path.join(super_sess_path, fn['structure']['ss_raw'])
super_file_path

'/mnt/cube/earneodo/bci_zf/ss_data/b1114/day-anesth_surface_depth_2017-03-10_16-30-53_450/experiment.raw.kwd'

In [55]:
reload(ss)
exp_list = ss.list_experiment_files(bird_id, sess_day, depth, raw_location=raw_location)

2017-03-12 21:10:58,788 supersession INFO     searching *_100.raw.kwd


In [56]:
reload(ss)

sess_list = ss.make_super_session(bird_id, sess_day, depth, raw_location=raw_location)

2017-03-12 21:11:01,698 supersession INFO     searching *_100.raw.kwd
2017-03-12 21:11:01,721 supersession INFO     Making supersession day-anesth_surface_depth_2017-03-10_16-30-53_450
2017-03-12 21:11:01,722 supersession INFO     super file path: /mnt/cube/earneodo/bci_zf/ss_data/b1114/day-anesth_surface_depth_2017-03-10_16-30-53_450/experiment.raw.kwd
2017-03-12 21:11:01,724 supersession INFO     Found 1 experiment files
2017-03-12 21:11:01,725 bci.core.expstruct DEBUG    Creating directory /mnt/cube/earneodo/bci_zf/ss_data/b1114/day-anesth_surface_depth_2017-03-10_16-30-53_450
2017-03-12 21:11:01,748 supersession INFO     Inserting file /mnt/cube/earneodo/bci_zf/raw_data/b1114/anesth_surface_depth_2017-03-10_16-30-53_450/experiment1_100.raw.kwd
2017-03-12 21:11:01,773 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r+)>


In [57]:
reload(et)
et.get_locations()

{'archive': '/mnt/cube/earneodo/archive/bci_zf',
 'experiment': '/mnt/cube/earneodo/bci_zf',
 'experiment_local': '/usr/local/experiment',
 'repos': '/mnt/cube/earneodo/repos',
 'scratch': '/usr/local/experiment/scratchpad',
 'store': '/Data/bci_zf'}

In [58]:
reload(ss)
ss.make_raw_bkp(bird_id, sess_list)

2017-03-12 21:22:11,095 supersession INFO     Data is already in experiment folder /mnt/cube/earneodo/bci_zf/raw_data/b1114
2017-03-12 21:22:11,098 supersession INFO     Data is not in local drive but in /mnt/cube/earneodo/bci_zf/raw_data/b1114, doing nothing


In [59]:
reload(ss)

ss.extract_wav_chans(bird_id, super_sess_name)

2017-03-12 21:22:11,131 supersession INFO     Extract mic chan to wav for recs [u'0']
2017-03-12 21:22:11,134 supersession INFO     Rec 0: /mnt/cube/earneodo/bci_zf/ss_data/b1114/day-anesth_surface_depth_2017-03-10_16-30-53_450/experiment-rec_000.mic.wav
2017-03-12 21:25:25,145 supersession INFO     Done extracting channels


In [60]:
sa.align_stim(bird_id, super_sess_name)

2017-03-12 21:25:25,161 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-03-12 21:25:25,163 root         DEBUG    H5 file: <HDF5 file "experiment.raw.kwd" (mode r)>
2017-03-12 21:25:25,164 root         DEBUG    Getting group of rec 0
2017-03-12 21:25:25,167 stimalign    DEBUG    Rec origin: {'rec': 0, 'sess': 'anesth_surface_depth_2017-03-10_16-30-53_450', 'bird': 'b1114', 'structure': 'experiment1_100'}
2017-03-12 21:25:25,263 stimalign    INFO     Getting starts of stim bos01-rev_tag
2017-03-12 21:25:25,266 root         DEBUG    H5 file: <HDF5 file "experiment1_100.raw.kwd" (mode r)>
2017-03-12 21:25:25,267 root         DEBUG    Getting dataset from rec 0
2017-03-12 21:36:49,250 stimalign    INFO     Found 75 starts
2017-03-12 21:36:49,273 stimalign    INFO     Stored in /mnt/cube/earneodo/bci_zf/ss_data/b1114/day-anesth_surface_depth_2017-03-10_16-30-53_450/experiment.sng.kwe
2017-03-12 21:36:49,275 stimalign    INFO     Getting starts of stim bos01_tag


In [6]:
bkp_dest_path = os.path.join(cube_experiment_folder, 'raw_data', bird_id)
store_dest_path = os.path.join(store_experiment_folder, 'raw_data', bird_id)

In [12]:
os.path.split(sessions[0])[0]==bkp_dest_path

True

In [36]:
fn

{'folders': {'kai': '/mnt/cube/kai/results/z030/day-anesth_surface_2016-12-16_0',
  'prb': '/mnt/cube/earneodo/bci_zf/probes',
  'raw': '/usr/local/experiment/raw_data/z030/day-anesth_surface_2016-12-16_0',
  'rw': '/mnt/cube/earneodo/bci_zf/raw_data/z030/day-anesth_surface_2016-12-16_0',
  'ss': '/mnt/cube/earneodo/bci_zf/ss_data/z030/day-anesth_surface_2016-12-16_0',
  'stim': '/mnt/cube/earneodo/bci_zf/stim_data/z030/day-anesth_surface_2016-12-16_0',
  'templ': '/mnt/cube/earneodo/bci_zf/templates',
  'tmp': '/usr/local/experiment/tmp_data/z030/day-anesth_surface_2016-12-16_0'},
 'locations': {'archive': '/mnt/cube/earneodo/archive/bci_zf',
  'experiment': '/mnt/cube/earneodo/bci_zf',
  'experiment_local': '/usr/local/experiment',
  'local_store': '/Data/bci_zf',
  'repos': '/mnt/cube/earneodo/repos'},
 'structure': {'cand': 'experiment.mot.h5',
  'evt': 'experiment.kwe',
  'kk_par': 'params.prm',
  'kk_prb': '*.prb',
  'ks_cfg': 'config.m',
  'ks_map': 'chanMap.mat',
  'ks_mas': 'm

In [56]:
evt_fn = et.file_names(bird_id, os.path.split(sessions[0])[-1], base='experiment1')
rec_ev_file_path = et.file_path(evt_fn, 'rw', 'evt')
with h5py.File(rec_ev_file_path, 'r') as rec_ev_file:
    s_l = sa.get_stims_list(rec_ev_file)

In [57]:
s_l

array(['bos_rev_tag', 'bos_syn_tag', 'bos_tag'], dtype=object)

In [51]:
et.file_names(bird_id, os.path.split(sessions[0])[-1], base='experiment1')

{'folders': {'kai': '/mnt/cube/kai/results/z030/anesth_surface_2016-12-16_12-27-11_0',
  'prb': '/mnt/cube/earneodo/bci_zf/probes',
  'raw': '/usr/local/experiment/raw_data/z030/anesth_surface_2016-12-16_12-27-11_0',
  'rw': '/mnt/cube/earneodo/bci_zf/raw_data/z030/anesth_surface_2016-12-16_12-27-11_0',
  'ss': '/mnt/cube/earneodo/bci_zf/ss_data/z030/anesth_surface_2016-12-16_12-27-11_0',
  'stim': '/mnt/cube/earneodo/bci_zf/stim_data/z030/anesth_surface_2016-12-16_12-27-11_0',
  'templ': '/mnt/cube/earneodo/bci_zf/templates',
  'tmp': '/usr/local/experiment/tmp_data/z030/anesth_surface_2016-12-16_12-27-11_0'},
 'locations': {'archive': '/mnt/cube/earneodo/archive/bci_zf',
  'experiment': '/mnt/cube/earneodo/bci_zf',
  'experiment_local': '/usr/local/experiment',
  'repos': '/mnt/cube/earneodo/repos',
  'store': '/Data/bci_zf'},
 'structure': {'cand': 'experiment1.mot.h5',
  'evt': 'experiment1.kwe',
  'kk_par': 'params.prm',
  'kk_prb': '*.prb',
  'ks_cfg': 'config.m',
  'ks_map': 'ch

In [53]:
with h5py.File(rec_ev_file_path, 'r') as rec_ev_file:
    sa.get_messages(rec_ev_file)

IOError: Unable to open file (Unable to open file: name = '/mnt/cube/earneodo/bci_zf/raw_data/z030/day-anesth_surface_2016-12-16_0/experiment.kwe', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)